In [1]:
import sys
sys.path.append('..')

from itertools import chain, combinations
from aimacode.planning import Action
from aimacode.utils import expr

from layers import BaseActionLayer, BaseLiteralLayer, makeNoOp, make_node

In [2]:
class ActionLayer(BaseActionLayer):

    def _inconsistent_effects(self, actionA, actionB):
        """ Return True if an effect of one action negates an effect of the other

        See Also
        --------
        layers.ActionNode
        """
        # TODO: implement this function
        raise NotImplementedError


    def _interference(self, actionA, actionB):
        """ Return True if the effects of either action negate the preconditions of the other 
        
        See Also
        --------
        layers.ActionNode
        """
        # TODO: implement this function
        raise NotImplementedError

    def _competing_needs(self, actionA, actionB):
        """ Return True if any preconditions of the two actions are pairwise mutex in the parent layer
        
        See Also
        --------
        layers.ActionNode
        layers.BaseLayer.parent_layer
        """
        # TODO: implement this function
        raise NotImplementedError

In [3]:
def _inconsistent_effects(self, actionA, actionB):
        """ Return True if an effect of one action negates an effect of the other

        See Also
        --------
        layers.ActionNode
        """
        # TODO: implement this function
        raise NotImplementedError

In [4]:
from layers import ActionNode

In [5]:
print(ActionNode.__doc__)

 Efficient representation of Actions for planning graph

    Attributes
    ----------
    expr : Expr
        An instance of aimacode.utils.Expr (a string-based symbolic expression)

    preconditions : set()
        A set of mixed positive and negative literal aimacode.utils.Expr
        expressions (symbolic representations like X, ~Y, etc.) that are
        preconditions of this action
        
    effects : set()
        A set of mixed positive and negative literal aimacode.utils.Expr
        expressions (symbolic representations like X, ~Y, etc.) that are
        results of applying this action

    no_op : bool
        A boolean flag indicating whether the instance is a no-op action
        (used to serialize planning graphs)
    


In [6]:
import example_have_cake as ex

In [7]:
cake_problem = ex.have_cake()

In [8]:
actions = cake_problem.get_actions()

In [9]:
type(actions[0])

aimacode.planning.Action

In [10]:
precond_pos = [expr("Have(Cake)")]
precond_neg = []
effect_add = [expr("Eaten(Cake)")]
effect_rem = [expr("Have(Cake)")]
eat_action = Action(expr("Eat(Cake)"),
                    [precond_pos, precond_neg],
                    [effect_add, effect_rem])

In [11]:
eat_action

In [12]:
preconditions = {expr("Have(Cake)")}
symbol = expr("Eat(Cake)")
effects = {expr("Eaten(Cake)"), expr("~Have(Cake)")}
a1 = ActionNode(symbol, preconditions, effects, no_op=False)

In [13]:
preconditions = {expr("~Have(Cake)")}
symbol = expr("Bake(Cake)")
effects = {expr("Have(Cake)")}
a2 = ActionNode(symbol, preconditions, effects, no_op=False)

In [14]:
a1

Eat(Cake)

In [15]:
a2

Bake(Cake)

In [16]:
a1.effects

{Eaten(Cake), ~Have(Cake)}

In [17]:
a2.effects

{Have(Cake)}

In [18]:
a1_iter = iter(a1.effects)
e = next(a1_iter)

In [19]:
e.__invert__()

Have(Cake)

In [20]:
e2 = next(iter(a2.effects))

In [21]:
e.__eq__(e2)

False

In [22]:
e1 = next(a1_iter)

In [23]:
a1.effects

{Eaten(Cake), ~Have(Cake)}

In [24]:
e1.__eq__(e2.__invert__())

False

In [25]:
e1

Eaten(Cake)

In [26]:
e2

Have(Cake)

In [27]:
from itertools import product

list(product(a1.effects, a2.effects))

[(~Have(Cake), Have(Cake)), (Eaten(Cake), Have(Cake))]

In [28]:
inconsistent = False
for e1, e2 in product(a1.effects, a2.effects):
    inconsistent = e1.__eq__(e2.__invert__())
    if inconsistent:
        break
inconsistent

True

In [29]:
def _inconsistent_effects(self, actionA, actionB):
    """ Return True if an effect of one action negates an effect of the other

    See Also
    --------
    layers.ActionNode
    """
    inconsistent = False
    for e1, e2 in product(actionA.effects, actionB.effects):
        inconsistent = e1.__eq__(e2.__invert__())
        if inconsistent:
            break
    return inconsistent

In [30]:
_inconsistent_effects(None, a1, a2)

True

### Interference

In [33]:
print(a1.preconditions)
print(a1.effects)
a1

{Have(Cake)}
{~Have(Cake), Eaten(Cake)}


Eat(Cake)

In [34]:
print(a2.preconditions)
print(a2.effects)
a2

{~Have(Cake)}
{Have(Cake)}


Bake(Cake)

a1 y a2 no interfieren, pero a2 interfiere consigo misma.

In [35]:
    def _interference(self, actionA, actionB):
        """ Return True if the effects of either action negate the preconditions of the other 
        
        See Also
        --------
        layers.ActionNode
        """
        inconsistent = False
        for p1, e2 in product(actionA.preconditions, actionB.effects):
            inconsistent = p1.__eq__(e2.__invert__())
            if inconsistent:
                return True
        for e1, p2 in product(actionA.effects, actionB.preconditions):
            inconsistent = e1.__eq__(p2.__invert__())
            if inconsistent:
                return True
        return inconsistent

In [37]:
_interference(None, a1,a2)

False

In [38]:
_interference(None, a2, a2)

True

### Competing needs

I won't simulate this because it would require to build a 2-layers graph and I think I can solve this function without so much work. If the first approach doesn't work I may do the simulation.

In [39]:
def _competing_needs(self, actionA, actionB):
        """ Return True if any preconditions of the two actions are pairwise 
        mutex in the parent layer
        
        See Also
        --------
        layers.ActionNode
        layers.BaseLayer.parent_layer
        """
        return self.parent_layer.is_mutex(actionA, actionB)

### The complete class

In [40]:
class ActionLayer(BaseActionLayer):

    def _inconsistent_effects(self, actionA, actionB):
        """ Return True if an effect of one action negates an effect of the other

        See Also
        --------
        layers.ActionNode
        """
        inconsistent = False
        for e1, e2 in product(actionA.effects, actionB.effects):
            inconsistent = e1.__eq__(e2.__invert__())
            if inconsistent:
                break
        return inconsistent


    def _interference(self, actionA, actionB):
        """ Return True if the effects of either action negate the preconditions of the other 
        
        See Also
        --------
        layers.ActionNode
        """
        inconsistent = False
        for p1, e2 in product(actionA.preconditions, actionB.effects):
            inconsistent = p1.__eq__(e2.__invert__())
            if inconsistent:
                return True
        for e1, p2 in product(actionA.effects, actionB.preconditions):
            inconsistent = e1.__eq__(p2.__invert__())
            if inconsistent:
                return True
        return inconsistent

    def _competing_needs(self, actionA, actionB):
        """ Return True if any preconditions of the two actions are pairwise mutex in the parent layer
        
        See Also
        --------
        layers.ActionNode
        layers.BaseLayer.parent_layer
        """
        return self.parent_layer.is_mutex(actionA, actionB)